# Train and Test an MLP classifier
In this notebook we reproduce results from [Ding et al., 2023](https://arxiv.org/pdf/2308.01402.pdf)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import pandas as pd
import numpy as np
import json
from argparse import ArgumentParser
import wandb
import random
from pytorch_lightning.callbacks import TQDMProgressBar
from sklearn.metrics import (
    balanced_accuracy_score,
    f1_score,
    log_loss,
    matthews_corrcoef,
    roc_auc_score,
)
try:
    from ogbcomp_helper import getmorganfingerprint2
except:
    pass

## Define Model Class
This code was grabbed directly from [Ding et. al, 2023](https://arxiv.org/abs/2308.01402)

In [2]:
# Load data using pandas, convert each line to dictionary
def load_data(filename):
    df = pd.read_csv(filename)
    df = df.to_dict(orient="records")
    return df


# Abstract fingerprint function
def fingerprint_func(smile):
    return getmorganfingerprint2(smile).to_numpy().tolist()


# Generate fingerprint for each molecule
def generate_fingerprint(data, fingerprint_func):
    smile2fingerprint = {}
    for item in data:
        for i in range(1, 5):  # 4 molecules for each experiment
            smile = item[f"m{i}"]
            if smile not in smile2fingerprint:
                smile2fingerprint[smile] = fingerprint_func(smile)
    return smile2fingerprint


# Convert value to one-hot vector given min, max, step
def convert_to_one_hot(val, min, max, step):
    result = []
    for i in range(int((max - min) / step)):
        if i * step <= val < (i + 1) * step:
            result.append(1)
        else:
            result.append(0)
    return result


# Define data loader
class DataModule(pl.LightningDataModule):
    def __init__(self, hparams):
        super(DataModule, self).__init__()
        self.smile2fingerprint = json.load(open(hparams.fingerprint_path))
        self.data = load_data(hparams.data_path)

        self.processed_data = []
        for item in self.data:
            result = {}

            result["label"] = int(item["y2"])
            psum = item["p1"] + item["p2"] + item["p3"]
            result["p1_feature"] = convert_to_one_hot(
                item["p1"] / psum, min=0, max=1, step=0.05
            )
            result["p2_feature"] = convert_to_one_hot(
                item["p2"] / psum, min=0, max=1, step=0.05
            )
            result["p3_feature"] = convert_to_one_hot(
                item["p3"] / psum, min=0, max=1, step=0.05
            )
            result["p4_feature"] = convert_to_one_hot(
                item["p4"], min=0, max=1.5, step=0.075
            )
            result["m1_fingerprint"] = self.smile2fingerprint[item["m1"]]
            result["m2_fingerprint"] = self.smile2fingerprint[item["m2"]]
            result["m3_fingerprint"] = self.smile2fingerprint[item["m3"]]
            result["m4_fingerprint"] = self.smile2fingerprint[item["m4"]]

            result["feature"] = torch.tensor(
                result["p1_feature"]
                + result["p2_feature"]
                + result["p3_feature"]
                + result["p4_feature"]
                + result["m1_fingerprint"]
                + result["m2_fingerprint"]
                + result["m3_fingerprint"]
                + result["m4_fingerprint"],
                dtype=torch.float,
            )

            self.processed_data.append(result)

        if "external_test" in hparams.data_path:
            self.test_data = self.processed_data
            print(len(self.test_data))
        else:
            # split data into training, validation and test set
            random.shuffle(self.processed_data)
            self.training_data = self.processed_data[
                : int(len(self.processed_data) * 0.8)
            ]
            self.val_data = self.processed_data[
                int(len(self.processed_data) * 0.8) : int(
                    len(self.processed_data) * 0.9
                )
            ]
            self.test_data = self.processed_data[int(len(self.processed_data) * 0.9) :]
            print(len(self.training_data), len(self.val_data), len(self.test_data))

        self.batch_size = hparams.batch_size

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.training_data, batch_size=self.batch_size, shuffle=True
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_data, batch_size=self.batch_size, shuffle=False
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_data, batch_size=self.batch_size, shuffle=False
        )


# Define model: 2-layer MLP
class MLP(pl.LightningModule):
    def __init__(self, hparams):
        super(MLP, self).__init__()
        self.num_features = hparams.feature_size * 4 + 20 * 4
        self.num_classes = 2
        self.learning_rate = hparams.learning_rate

        self.layer1 = nn.Linear(self.num_features, hparams.hidden_dim)
        self.layer2 = nn.Linear(hparams.hidden_dim, self.num_classes)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        return x

    def training_step(self, batch, batch_idx):
        y_hat = self.forward(batch["feature"])
        loss = F.cross_entropy(y_hat, batch["label"])
        f1 = f1_score(
            batch["label"].detach().numpy(), y_hat.detach().numpy().argmax(axis=1)
        )
        try:
            auc_roc = roc_auc_score(
                batch["label"].detach().numpy(), y_hat.detach().numpy()[:, 1]
            )
        except:
            auc_roc = 0

        self.log_dict({"train/loss": loss, "train/f1": f1, "train/auc_roc": auc_roc})
        return {"loss": loss, "f1": f1, "auc_roc": auc_roc}

    def validation_step(self, batch, batch_idx):
        y_hat = self.forward(batch["feature"])
        loss = F.cross_entropy(y_hat, batch["label"])
        return {"loss": loss, "score": y_hat, "label": batch["label"]}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
        score = torch.cat([x["score"] for x in outputs]).numpy()
        pred = score.argmax(axis=1)
        label = torch.cat([x["label"] for x in outputs]).numpy()
        f1 = f1_score(label, pred)
        auc_roc = roc_auc_score(label, score[:, 1])
        mcc = matthews_corrcoef(label, pred)
        balanced_accuracy = balanced_accuracy_score(label, pred)
        self.log_dict({"val/loss": avg_loss, "val/f1": f1, "val/auc_roc": auc_roc, 'val/balanced_accuracy': balanced_accuracy, 'val/mcc': mcc})
        return {"val/loss": avg_loss, "val/f1": f1, "val/auc_roc": auc_roc, 'val/balanced_accuracy': balanced_accuracy, 'val/mcc': mcc}

    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
        score = torch.cat([x["score"] for x in outputs]).numpy()
        pred = score.argmax(axis=1)
        label = torch.cat([x["label"] for x in outputs]).numpy()
        f1 = f1_score(label, pred)
        auc_roc = roc_auc_score(label, score[:, 1])
        # self.logger.experiment.log(
        #     {
        #         "conf_mat": wandb.plot.confusion_matrix(
        #             probs=None, y_true=label, preds=pred, class_names=["0", "1"]
        #         )
        #     }
        # )
        mcc = matthews_corrcoef(label, pred)
        balanced_accuracy = balanced_accuracy_score(label, pred)
        self.log_dict({"test/loss": avg_loss, "test/f1": f1, "test/auc_roc": auc_roc, 'test/balanced_accuracy': balanced_accuracy, 'test/mcc':mcc})
        return {"test/loss": avg_loss, "test/f1": f1, "test/auc_roc": auc_roc, 'test/balanced_accuracy': balanced_accuracy, 'test/mcc': mcc}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)


## Load data

Download data and fingerprints from [here](https://anonymous.4open.science/r/Lipid_Nanoparticle_Design/data/all_data.csv)

In [3]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [4]:
def train_mlp(fingerprint_path, feature_size):
    # Simulating argparse parsing
    args = Namespace(
        data_path="../data/ding_et_al/all_data.csv",
        fingerprint_path=fingerprint_path,
        hidden_dim=128,
        batch_size=32,
        num_epochs=1000,
        learning_rate=0.001,
        seed=42,
        group_id=0,
        feature_size=feature_size
    )
    np.random.seed(args.seed)
    random.seed(args.seed)

    # logger = pl.loggers.WandbLogger(project="mRNA", config=vars(args), log_model=False)
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        save_top_k=1, monitor="val/auc_roc", mode="max"
    )
    # lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval="step")

    model = MLP(hparams=args)
    data_module = DataModule(hparams=args)
    trainer = pl.Trainer(
        gpus=0,
        max_epochs=args.num_epochs,
        callbacks=[checkpoint_callback,
            TQDMProgressBar(refresh_rate=100)],
    )
    trainer.fit(model, datamodule=data_module)
    trainer.validate(datamodule=data_module)
    trainer.test(datamodule=data_module)

    return trainer, data_module


In [5]:
# trainer_expert, dm_expert = train_mlp('../data/ding_et_al/mol2fp_expert.json', 200)

In [6]:
trainer_grover, dm_grover = train_mlp('../data/ding_et_al/mol2fp_grover.json', 3400)

/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:467: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` pa

497 62 63


Sanity Checking: 0it [00:00, ?it/s]

/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1600: PossibleUserWarning: The number of training 

Training: 0it [00:00, ?it/s]

...

/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:124: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at /home/oneai/code/LipoBART/notebooks/lightning_logs/version_26/checkpoints/epoch=953-step=15264.ckpt
Loaded model weights from checkpoint at /home/oneai/code/LipoBART/notebooks/lightning_logs/version_26/checkpoints/epoch=953-step=15264.ckpt


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       val/auc_roc          0.8624113475177305
  val/balanced_accuracy      0.801418439716312
         val/f1             0.7142857142857142
        val/loss             0.342743456363678
         val/mcc            0.6341976893663289
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test/auc_roc                0.9375
 test/balanced_accuracy     0.9162234042553192
         test/f1                   0.875
        test/loss           0.2246996909379959
        test/mcc            0.8324468085106383
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [7]:
trainer_expert_grover, dm_expert_grover = train_mlp('../data/ding_et_al/mol2fp_expert_grover.json', 3600)

/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:467: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name   | Type   | Params
----------------------------------
0 | layer1 | Linear | 1.9 M 
1 | layer2 | Linear | 258   
----------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
7.415     Total estima

497 62 63


Sanity Checking: 0it [00:00, ?it/s]

/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/oneai/storage/mambaforge/envs/ding2/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1600: PossibleUserWarning: The number of training 

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]